In [1]:
# Import Libraries
import pandas as pd
import pickle

In [2]:
# pip install -r requirements.txt

In [3]:
# pip install tensorflow

In [4]:
# pip install transformers

In [5]:
# pip install torch

In [6]:
# Globals for automatic nature
MAKE_DATA = True
ROWS = 100

In [7]:
# Load data
from data.data import Data

if MAKE_DATA:
    data_raw = Data().get_all_data()
    df = data_raw['MBTI 500']

    df = df[['type', 'posts']]
    df.rename(columns={'posts': 'text'}, inplace=True)
    df = df.sample(n = ROWS, ignore_index = True)

    print(f"MBTI_500 loaded with shape: {df.shape}")


MBTI_500 loaded with shape: (100, 2)


In [8]:
# Bert Preprocessing
from scripts.BERT_preprocessing import training_preprocessing

df_list = training_preprocessing(df)
df_list[0]

/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a mode

cleaned dataset contains 100 rows and 6 columns
Processed 1/599 samples
Processed 2/599 samples
Processed 3/599 samples
Processed 4/599 samples
Processed 5/599 samples
Processed 6/599 samples
Processed 7/599 samples
Processed 8/599 samples
Processed 9/599 samples
Processed 10/599 samples
Processed 11/599 samples
Processed 12/599 samples
Processed 13/599 samples
Processed 14/599 samples
Processed 15/599 samples
Processed 16/599 samples
Processed 17/599 samples
Processed 18/599 samples
Processed 19/599 samples
Processed 20/599 samples
Processed 21/599 samples
Processed 22/599 samples
Processed 23/599 samples
Processed 24/599 samples
Processed 25/599 samples
Processed 26/599 samples
Processed 27/599 samples
Processed 28/599 samples
Processed 29/599 samples
Processed 30/599 samples
Processed 31/599 samples
Processed 32/599 samples
Processed 33/599 samples
Processed 34/599 samples
Processed 35/599 samples
Processed 36/599 samples
Processed 37/599 samples
Processed 38/599 samples
Processed 3

,type,embeddings
0,I,"[[-0.09164603799581528, -0.20378568768501282, ..."
1,I,"[[-0.2331148386001587, -0.30213338136672974, 0..."
2,I,"[[-0.10865837335586548, -0.2094014436006546, 0..."
3,I,"[[0.1399361938238144, -0.1437709778547287, 0.3..."
4,I,"[[-0.30207201838493347, 0.11711004376411438, 0..."
...,...,...
353,E,"[[-0.005390668287873268, 0.10123320668935776, ..."
354,E,"[[-0.24849100410938263, -0.13963694870471954, ..."
355,E,"[[-0.061292119324207306, -0.042858511209487915..."
356,E,"[[-0.058733392506837845, 0.256392240524292, 0...."


In [9]:
# df_list[0].embeddings[64].shape

In [10]:
# Create or Load Models
from model.DL import *

model_list = initialize_models(metrics='accuracy',
                               loss='binary_crossentropy',
                               vocab_size=768)

2023-06-09 20:28:21.045403: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 20:28:22.057260: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-09 20:28:23.505819: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-09 20:28:23.506114: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the requir

In [11]:
model_list[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 768, 768)          590592    
                                                                 
 conv1d (Conv1D)             (None, 768, 10)           153610    
                                                                 
 conv1d_1 (Conv1D)           (None, 768, 10)           1010      
                                                                 
 flatten (Flatten)           (None, 7680)              0         
                                                                 
 dense (Dense)               (None, 20)                153620    
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 2

In [12]:
# Train Model

train_models(df_list, model_list)

/home/aforbesj/code/aforbesj/post2personality/model/DL.py:53: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_pad = np.asarray(X_pad).astype(np.float32)


ValueError: setting an array element with a sequence.

In [ ]:
# Predict Model